# Water Table Analysis
Author: Ian Coleman <br>
Purpose: Analyse water table data

In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import subprocess
import math
import seaborn as sb
import pdb

### Import Data

In [4]:
# # I need to invest in a function to intelligently determine type

# categorical_cols = ["extraction_type", "extraction_type_group", "extraction_type_class", 
#   "management", "management_group", "payment", "payment_type", "water_quality", 
#   "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", 
#   "waterpoint_type", "waterpoint_type_group"]


In [5]:
# df_cd = pd.read_csv('../ctd-to-nt/csvs/CTD_chemicals_diseases.csv', usecols=cds_cols, dtype=cd_col_types, skiprows=27)

In [6]:
# Import a subset so we can check appropriate dtypes and then import full dataset in a mem-efficient way 
# Though I wouldn't usually do this for a dataset of this size but want to show my general process
wt = pd.read_csv("data.csv")

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
## TODO
# Strip all cols
# Drop NA
# Address unknowns and similar
# force int cols to int type etc
# break col types further into uint, int16, int8 etc
# force uints to be uints BEWARE of conversion as methods like astype can mislead

In [8]:
# Iterate through columns, assign them to a data type
# NB the output will need to be checked by some common sense  

cat_cols = []
int_cols = []
float_cols = []
bool_cols = []
undetermined = []

for col in wt.columns:
    clean_col = wt[col].replace(0, np.nan)
    clean_col = wt[col].replace(0.0, np.nan)
    clean_col = wt[col].dropna()
    mode_val = str(clean_col.mode().values[0]).lower()
    
    if mode_val == 'true' or mode_val == 'false':
        bool_cols.append(col)
    elif isinstance(clean_col.mode().values[0], float):
        float_cols.append(col)
    elif mode_val.isnumeric():
        int_cols.append(col)
    elif (len(clean_col.unique()) < 100):
        cat_cols.append(col)
    else:
        undetermined.append(col)
        
        
print("\n int_cols --  \n", int_cols)
print("\n float_cols --  \n", float_cols)
print("\n cat_cols --  \n", cat_cols)
print("\n bool_cols --  \n", bool_cols)
print("\n undetermined --  \n", undetermined)


 int_cols --  
 ['id', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']

 float_cols --  
 ['amount_tsh', 'gps_height', 'longitude', 'latitude']

 cat_cols --  
 ['basin', 'region', 'recorded_by', 'scheme_management', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']

 bool_cols --  
 ['public_meeting', 'permit']

 undetermined --  
 ['date_recorded', 'funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_name']


In [9]:
# Set and check date type cols
print(wt["date_recorded"][0])
print(pd.to_datetime(wt["date_recorded"])[0])

3/14/11
2011-03-14 00:00:00


In [10]:
cat_cols = {key: "category" for key in cat_cols}
int_cols = {key: "int" for key in int_cols}
float_cols = {key: "float" for key in float_cols}
bool_cols = {key: "bool" for key in bool_cols} 
undetermined = {key: "object" for key in undetermined}
col_types = {**cat_cols, **int_cols, **float_cols, **bool_cols}

In [11]:
# # Enter the manual changes
col_types['date_recorded'] = 'datetime64'
# del col_types['date_recorded']

In [12]:
col_types

{'basin': 'category',
 'region': 'category',
 'recorded_by': 'category',
 'scheme_management': 'category',
 'extraction_type': 'category',
 'extraction_type_group': 'category',
 'extraction_type_class': 'category',
 'management': 'category',
 'management_group': 'category',
 'payment': 'category',
 'payment_type': 'category',
 'water_quality': 'category',
 'quality_group': 'category',
 'quantity': 'category',
 'quantity_group': 'category',
 'source': 'category',
 'source_type': 'category',
 'source_class': 'category',
 'waterpoint_type': 'category',
 'waterpoint_type_group': 'category',
 'id': 'int',
 'num_private': 'int',
 'region_code': 'int',
 'district_code': 'int',
 'population': 'int',
 'construction_year': 'int',
 'amount_tsh': 'float',
 'gps_height': 'float',
 'longitude': 'float',
 'latitude': 'float',
 'public_meeting': 'bool',
 'permit': 'bool',
 'date_recorded': 'datetime64'}

In [13]:
for col in col_types:
    wt[col] = wt[col].astype(col_types[col], errors='ignore')

In [14]:
# In this cell I sweep through possible errors using astype conversion without ignoring errors, 
# Then I patch each error and rerun until it runs through. Handy way to catch edge cases
for col in float_cols:
    wt[col] = wt[col].apply(lambda x: x if isinstance(x, float) else np.nan)

for col in int_cols:
    wt[col] = wt[col].apply(lambda x: x if isinstance(x, int) else np.nan)
    wt[col] = wt[col].replace([np.inf, -np.inf], np.nan)
#     wt[col] = wt[col].replace(np.nan, 0) ->

for col in col_types:
    wt[col] = wt[col].astype(col_types[col])

In [15]:
wt.dtypes

id                                int64
amount_tsh                      float64
date_recorded            datetime64[ns]
funder                           object
gps_height                      float64
installer                        object
longitude                       float64
latitude                        float64
wpt_name                         object
num_private                       int64
basin                          category
subvillage                       object
region                         category
region_code                       int64
district_code                     int64
lga                              object
ward                             object
population                        int64
public_meeting                     bool
recorded_by                    category
scheme_management              category
scheme_name                      object
permit                             bool
construction_year                 int64
extraction_type                category


In [16]:
wt.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0.0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [17]:
labels = pd.read_csv("labels.csv")
print('Shape: ', labels.shape)

Shape:  (59400, 2)


In [18]:
labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [19]:
labels['id'] = labels['id'].replace(to_replace ='[!]', value = np.nan, regex = True) # aimed at specific error
labels['id'].dropna(inplace=True)
labels['id'] = labels['id'].astype(int)
labels['status_group'] = labels['status_group'].astype('category')

In [20]:
wt = wt.merge(labels, on='id', how='left')

### Exploratory Analysis

In [21]:
wt.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0.0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [22]:
wt.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59402.000000,26634.000000,59401.000000,59402.000000,5.940200e+04,59402.000000,59402.000000,59402.000000,59402.000000,59402.000000
mean,37115.198209,328.704545,668.299069,34.077513,-5.706111e+00,0.474125,15.296724,5.629642,179.902007,1300.675381
std,21453.555072,3284.004629,693.107111,6.567348,2.946057e+00,12.236024,17.587185,9.633504,471.476973,951.612717
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,-200.000000,0.000000
25%,18519.250000,0.000000,0.000000,33.090410,-8.540946e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55657.500000,20.000000,1319.000000,37.178460,-3.326168e+00,0.000000,17.000000,5.000000,214.750000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [23]:
wt.dtypes

id                                int64
amount_tsh                      float64
date_recorded            datetime64[ns]
funder                           object
gps_height                      float64
installer                        object
longitude                       float64
latitude                        float64
wpt_name                         object
num_private                       int64
basin                          category
subvillage                       object
region                         category
region_code                       int64
district_code                     int64
lga                              object
ward                             object
population                        int64
public_meeting                     bool
recorded_by                    category
scheme_management              category
scheme_name                      object
permit                             bool
construction_year                 int64
extraction_type                category


In [24]:
wt.isna().sum()

id                           0
amount_tsh               32768
date_recorded                0
funder                    3635
gps_height                   1
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting               0
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                       0
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [ ]:
numeric

In [27]:
sb.pairplot(wt[[int_cols.keys() + float_cols.keys()]])

TypeError: unsupported operand type(s) for +: 'dict_keys' and 'dict_keys'